In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualization_utils as vis_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix,MeshcatVisualizer, StartMeshcat
import pydrake.symbolic as sym
from pydrake.solvers import MosekSolver, CommonSolverOption, SolverOptions, ScsSolver
from pydrake.all import PointCloud, MeshcatVisualizerParams, Role, HalfSpace, CoulombFriction, Box, Rgba, WeldJoint, GeometrySet
from pydrake.polynomial import Polynomial as PolynomialCommon
import time
from scipy.spatial.transform import Rotation as sp_rot
from sampling_based_motion_planners import StraightLineCollisionChecker, PRM, BiRRT, PRMFixedEdges


from pydrake.all import (RationalForwardKinematics, ModelInstanceIndex,
                         SpatialInertia, RevoluteJoint, FixedOffsetFrame, MultibodyPlant)
from pydrake.geometry.optimization import HPolyhedron, Hyperellipsoid
from pydrake.geometry.optimization_dev import CspaceFreePath


In [4]:
import logging
drake_logger = logging.getLogger("drake")
drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [5]:
meshcat = StartMeshcat()
meshcat.Delete()

DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not set.
DEBUG:drake:FindRunfile mechanism = RUNFILES_{MANIFEST_FILE,DIR}
DEBUG:drake:cwd = "/home/amice/Documents/coding_projects/drake/C_Iris_Examples"
DEBUG:drake:FindRunfile found by-manifest '/home/amice/Documents/coding_projects/drake/.drake-find_resource-sentinel' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/n_link_pend.runfiles/drake/.drake-find_resource-sentinel' (good)
DEBUG:drake:FindRunfile found by-manifest '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/geometry/meshcat.js' (good) and by-directory '/home/amice/.cache/bazel/_bazel_amice/32fdbbecfa8a7ce8feece95e48c42006/execroot/drake/bazel-out/k8-opt/bin/C_Iris_Examples/n_link_pend.runfiles/drake/geometry/meshcat.js' (good)
DEBUG:drake:FindResource ignoring DRAKE_RESOURCE_ROOT because it is not s

In [6]:

def AddPendulumLink(
    plant,
    pendulum_model_instance,
    link_dimensions,
    parent_frame,
    link_index: int,
    color=np.array([1, 0, 0, 1]),
):
    pendulum_box = Box(*link_dimensions)
    pendulum_inertia = SpatialInertia.SolidBoxWithDensity(1.0, *link_dimensions)

    link_name = f"link_{link_index}"
    pend_body = plant.AddRigidBody(link_name, pendulum_model_instance, pendulum_inertia)
    parent_to_child_attach_frame = plant.AddFrame(
        FixedOffsetFrame(
            f"{link_name}_joint_frame",
            pend_body.body_frame(),
            RigidTransform(p=(0, link_dimensions[1] / 2, 0)),
        )
    )
    next_frame = plant.AddFrame(
        FixedOffsetFrame(
            f"{link_name}_joint_frame",
            pend_body.body_frame(),
            RigidTransform(p=(0, -link_dimensions[1] / 2, 0)),
        )
    )
    # add the revolute joint,
    joint_lim = np.pi
    plant.AddJoint(
        RevoluteJoint(
            f"joint_{link_index}",
            parent_frame,
            parent_to_child_attach_frame,
            [0, 0, 1],
            -joint_lim,
            joint_lim,
        )
    )

    geom_id = plant.RegisterCollisionGeometry(
        pend_body, RigidTransform(), pendulum_box, link_name, CoulombFriction()
    )
    plant.RegisterVisualGeometry(
        pend_body, RigidTransform(), pendulum_box, link_name, color
    )

    return next_frame, geom_id


def N_Link_Pendulum(
    n: int,
    plant: MultibodyPlant,
    pendulum_model_instance: ModelInstanceIndex,
    link_dimensions: np.ndarray,
):
    assert n > 0
    parent_frame = plant.world_frame()
    colors = np.array(vis_utils.n_colors(n)) / 255
    colors = np.hstack([colors, np.ones((colors.shape[0], 1))])
    geom_ids = []
    for i in range(n):
        parent_frame, geom_id = AddPendulumLink(
            plant,
            pendulum_model_instance,
            link_dimensions,
            parent_frame,
            i + 1,
            colors[i],
        )
        geom_ids.append(geom_id)

    return geom_ids


def AddRandomBox(
    plant: MultibodyPlant,
    obstacle_model_instance: ModelInstanceIndex,
    index: int,
    pos_limits,
    size_limits,
    color=[0, 0, 0, 1],
):
    link_name = f"obstacle_{index}"

    center = np.append(np.random.uniform(pos_limits[0], pos_limits[1], 2), 0)

    

    dimensions = np.append(np.random.uniform(size_limits[0], size_limits[1], 2), 0.001)
    box = Box(*dimensions)
    inertia = SpatialInertia.SolidBoxWithDensity(1, *dimensions)

    body = plant.AddRigidBody(link_name, obstacle_model_instance, inertia)
    
    random_rp = np.zeros(3)
    lim = np.pi / 10
    random_rp[2] = np.random.uniform(-lim, lim)
    angle = RotationMatrix(RollPitchYaw(random_rp))
    pose = RigidTransform(p=center) @ RigidTransform(R=angle)
    origin_local = pose.inverse()@RigidTransform()
    while np.all([-dimensions[i]/2 <= origin_local.translation()[i] < dimensions[i]/2 for i in range(3)]): 
        random_rp[2] = np.random.uniform(-lim, lim)
        angle = RotationMatrix(RollPitchYaw(random_rp))
        pose = RigidTransform(p=center)@RigidTransform(R=angle)
        origin_local = pose.inverse()@RigidTransform()
        
    plant.AddJoint(WeldJoint(link_name, plant.world_frame(), body.body_frame(), pose))
    plant.RegisterCollisionGeometry(
        body, RigidTransform(), box, link_name, CoulombFriction()
    )
    plant.RegisterVisualGeometry(body, RigidTransform(), box, link_name, color)


def AddNRandomBoxes(
    n: int,
    plant: MultibodyPlant,
    obstacle_model_instance: ModelInstanceIndex,
    pos_limits=(-10, 10),
    size_limits=(0.1, 1),
    color=[0, 0, 0, 1],
):
    assert n > 0
    for i in range(n):
        AddRandomBox(plant, obstacle_model_instance, i, pos_limits, size_limits, color)


def make_line_polys(plant: MultibodyPlant, prm):
    polys = np.empty(shape=(plant.num_positions(), len(prm.prm.edges())), dtype=object)
    for i, (s0, s1) in enumerate(prm.prm.edges()):
        for j in range(plant.num_positions()):
            polys[j, i] = PolynomialCommon(np.array([s0[j], s1[j] - s0[j]]))
    return polys


def build_n_link_k_boxes_plant_and_certifier(
    n: int, k: int, exclude_pend_self_collisions=True, plane_order=1, maximum_path_degree=1
):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

    link_dimensions = (0.1, 0.2, 0.001)
    pendulum_model_instance = plant.AddModelInstance("pendulum")
    obstacle_model_instance = plant.AddModelInstance("obstacle")

    num_pend = n
    num_obstacles = k
    np.random.seed(num_pend * num_obstacles)

    pend_geom_ids = N_Link_Pendulum(
        num_pend, plant, pendulum_model_instance, np.array(link_dimensions)
    )
    
    if k > 0:
        obstacle_pos_limits = link_dimensions[1] * num_pend * np.array([-1.5, 1.5])
        obstacle_size_limits = (0.01 / np.sqrt(num_obstacles), 0.5 / np.sqrt(num_obstacles))
        AddNRandomBoxes(
            num_obstacles,
            plant,
            obstacle_model_instance,
            obstacle_pos_limits,
            obstacle_size_limits,
        )

    plant.Finalize()
    if exclude_pend_self_collisions:
        collision_filter_manager = scene_graph.collision_filter_manager()
        pend_geom_set = GeometrySet(pend_geom_ids)
        decl = CollisionFilterDeclaration().ExcludeWithin(pend_geom_set)
        collision_filter_manager.Apply(decl)
    
    visualizer = MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)
    
    diagram = builder.Build()

    q_star = np.zeros(plant.num_positions())
    t0 = time.time()
    cspace_free_path = CspaceFreePath(
        plant, scene_graph, q_star, maximum_path_degree=maximum_path_degree, plane_order=plane_order
    )
#     cspace_free_path = None
    t1 = time.time()
    print(f"Time to construct line certifier for {n}-links, {k}-boxes = {t1 - t0}s")
    return plant, scene_graph, diagram, cspace_free_path


In [10]:
n = 6
plane_order = 3
maximum_path_degree = 5
k = 1
plant, scene_graph, diagram, cspace_free_path = build_n_link_k_boxes_plant_and_certifier(n,k,exclude_pend_self_collisions = False,
                                                                                        plane_order=plane_order,
                                                                                        maximum_path_degree=maximum_path_degree)
diagram_context = diagram.CreateDefaultContext()
diagram.ForcedPublish(diagram_context)

Time to construct line certifier for 6-links, 1-boxes = 50.598339557647705s


In [11]:
for plane_geom in cspace_free_path.plane_geometries_on_path():
    max_deg = 0
    for c in plane_geom.positive_side_conditions + plane_geom.negative_side_conditions:
        max_deg = max(max_deg, c.get_p().TotalDegree())
    print(max_deg)
    print()

13

23

23

33

13

13

23

23

13

13

23

23

13

23

33

33



In [32]:
# t1 = plane_geom.positive_side_conditions[-1].get_p().monomial_to_coefficient_map()[sym.Monomial()]
print(plane_geom.positive_side_conditions[-1].get_psatz_variables_and_psd_constraints())

TypeError: Unable to convert function return value to a Python type! The signature was
	(self: pydrake.geometry.optimization_dev.ParametrizedPolynomialPositiveOnUnitInterval) -> drake::copyable_unique_ptr<drake::solvers::MathematicalProgram>

In [14]:
for plane in cspace_free_path.separating_planes():
    print(plane.a[0].TotalDegree())
    print()

3

3

3

3

3

3

3

3

3

3

3

3

3

3

3

3



In [11]:
q_star = np.zeros(plant.num_positions())
Ratfk = RationalForwardKinematics(plant)
diagram_col_context = diagram.CreateDefaultContext()
plant_col_context = diagram.GetMutableSubsystemContext(plant, diagram_col_context)
scene_graph_col_context = diagram.GetMutableSubsystemContext(
        scene_graph, diagram_col_context
    )
query_port = scene_graph.get_query_output_port()

def check_collision_q_by_query(q):
    if np.all(q >= plant.GetPositionLowerLimits()) and np.all(
        q <= plant.GetPositionUpperLimits()
    ):
        plant.SetPositions(plant_col_context, q)
        query_object = query_port.Eval(scene_graph_col_context)
        return 1 if query_object.HasCollisions() else 0
    else:
        return 1

def check_collision_s_by_query(s):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_query(q)

def sample_col_free_point():
    q = np.random.uniform(
        plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()
    )
    s = Ratfk.ComputeSValue(q, q_star)
    return s
l = 200
collision_checker = StraightLineCollisionChecker(
                    check_collision_s_by_query, 100
                )
prm = PRMFixedEdges(
                sample_col_free_point, l, collision_checker, dist_thresh=100
            )
diagram_vis_context = diagram.CreateDefaultContext()
plant_vis_context = diagram.GetMutableSubsystemContext(plant, diagram_vis_context)
vis_bundle = vis_utils.VisualizationBundle(
    diagram, diagram_context, plant, plant_vis_context,
    Ratfk, meshcat, q_star
)
end_effector = plant.GetBodyByName(f"link_{n}")

prm.draw_tree(vis_bundle, end_effector)

PRM has 14 nodes
PRM has 33 edges
PRM has 28 nodes
PRM has 81 edges
PRM has 42 nodes
PRM has 126 edges
PRM has 56 nodes
PRM has 186 edges
PRM has 70 nodes
PRM has 200 edges
PRM has 70 nodes
PRM has 200 edges


In [12]:
def make_line_polys(prm, max_num_edges = -1):
    
    polys = np.empty(shape=(plant.num_positions(), 
                            len(prm.prm.edges()) if max_num_edges < 0 else max_num_edges), dtype = object)
    for i, (s0, s1) in enumerate(prm.prm.edges()):
        for j in range(plant.num_positions()):
            if max_num_edges > 0 and not i < max_num_edges:
                break
            polys[j,i] = PolynomialCommon(np.array([s0[j],s1[j]-s0[j]]))
        
    return polys

path_safe = make_line_polys(prm)
# path_unsafe = make_line_polys(PRM_bad)
    

In [13]:
cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False

cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False

t0 = time.time()
statistics, cert_result = \
    cspace_free_path.FindSeparationCertificateGivenPath(path_safe, 
                                                        set(),
                                                        cert_options)
t1 = time.time()
print(f"Certification of safe PRM in {t1-t0}s")

Certification of safe PRM in 3.3198323249816895s
ENTERING METHOD
FUNCTION IS RETURNING


In [ ]:
meshcat.Delete()
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)

visualizer = MeshcatVisualizer.AddToBuilder(
    builder, scene_graph, meshcat)

link_dimensions = (0.1, 0.2, 0.001)
pendulum_box = Box(*link_dimensions)
pendulum_inertia = SpatialInertia.SolidBoxWithDensity(1, *link_dimensions)
pendulum_model_instance = plant.AddModelInstance("pendulum")
def AddPendulumLink(parent_frame, link_index: int, color = np.array([1,0,0,1])):
    link_name = f"link_{link_index}"
    pend_body = plant.AddRigidBody(link_name, pendulum_model_instance, pendulum_inertia)
    parent_to_child_attach_frame = plant.AddFrame(
        FixedOffsetFrame(f"{link_name}_joint_frame", pend_body.body_frame(), 
                                  RigidTransform(p = (0,link_dimensions[1]/2, 0)))
    )
    next_frame = plant.AddFrame(
        FixedOffsetFrame(f"{link_name}_joint_frame", pend_body.body_frame(), 
                                  RigidTransform(p = (0,-link_dimensions[1]/2, 0)))
    )
    # add the revolute joint,
    joint_lim = np.pi
    plant.AddJoint(RevoluteJoint(
        f"joint_{link_index}", parent_frame, parent_to_child_attach_frame, [0,0,1],
        -joint_lim, joint_lim,
    )
    )
    
    geom_id = plant.RegisterCollisionGeometry(pend_body, RigidTransform(),
                                    pendulum_box, link_name, CoulombFriction())
    plant.RegisterVisualGeometry(pend_body, RigidTransform(),
                                pendulum_box, link_name, color)
    
    
    
    return next_frame, geom_id

def N_Link_Pendulum(n: int):
    assert n > 0
    parent_frame = plant.world_frame()
    colors = np.array(vis_utils.n_colors(n))/255
    colors = np.hstack([colors,np.ones((colors.shape[0],1))])
    geom_ids = []
    for i in range(n):
        parent_frame, geom_id = AddPendulumLink(parent_frame, i+1, colors[i])
        geom_ids.append(geom_id)

    return geom_ids
        
obstacle_model_instance = plant.AddModelInstance("obstacle")
def AddRandomBox(index: int, pos_limits, size_limits, color = [0,0,0,1]):
    link_name = f"obstacle_{index}"     
    
    center = np.append(np.random.uniform(pos_limits[0],pos_limits[1], 2),0)
        
    dimensions = np.append(np.random.uniform(size_limits[0],size_limits[1],2),0.001)
    box = Box(*dimensions)
    inertia = SpatialInertia.SolidBoxWithDensity(1, *dimensions)
    
    body = plant.AddRigidBody(link_name, obstacle_model_instance, inertia)
    
    random_rp = np.zeros(3)
    lim = np.pi/10
    random_rp[2] = np.random.uniform(-lim, lim)
    angle = RotationMatrix(RollPitchYaw(random_rp))
    pose = RigidTransform(p=center)@RigidTransform(R=angle)
    origin_local = pose.inverse()@RigidTransform()
    while np.all([-dimensions[i]/2 <= origin_local.translation()[i] < dimensions[i]/2 for i in range(3)]): 
        random_rp[2] = np.random.uniform(-lim, lim)
        angle = RotationMatrix(RollPitchYaw(random_rp))
        pose = RigidTransform(p=center)@RigidTransform(R=angle)
        origin_local = pose.inverse()@RigidTransform()
    plant.AddJoint(WeldJoint(link_name, plant.world_frame(), body.body_frame(),
                            pose))
    plant.RegisterCollisionGeometry(body, RigidTransform(),
                                    box, link_name, CoulombFriction())
    plant.RegisterVisualGeometry(body, RigidTransform(),
                                box, link_name, color)
    
    

def AddNRandomBoxes(n: int, pos_limits=(-10,10),
                    size_limits = (0.1,1), color= [0,0,0,1],
                   filter_all = False):
    assert n > 0
    for i in range(n):
        AddRandomBox(i, pos_limits, size_limits, color)

# child_frame = AddPendulumLink(plant.world_frame(), 1)
num_pend = 3
num_obstacles = 4
np.random.seed(num_pend * num_obstacles)


pend_geom_ids = N_Link_Pendulum(num_pend)

obstacle_pos_limits = link_dimensions[1]*num_pend*np.array([-1,1])
obstacle_size_limits = (0.01/np.log(num_obstacles),0.5/np.log(num_obstacles))
AddNRandomBoxes(num_obstacles, obstacle_pos_limits, obstacle_size_limits)


plant.Finalize()

end_effector = plant.GetBodyByName(f"link_{num_pend}")

def SetMeshcatPerspective2d(meshcat_instance, X_WC):
    meshcat_instance.SetTransform("/Cameras/default", X_WC)
    meshcat_instance.SetProperty("/Background", "visible", False);
    meshcat_instance.SetProperty("/Grid", "visible", False);
    meshcat_instance.SetProperty("/Axes", "visible", False)

SetMeshcatPerspective2d(meshcat, RigidTransform(p=np.array([0,0,1])))
# meshcat.Set2dRenderMode(RigidTransform(p=np.array([0,0,1])))

diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
scene_graph_context = diagram.GetMutableSubsystemContext(scene_graph, diagram_context)
query_port = scene_graph.get_query_output_port()
diagram.ForcedPublish(diagram_context)

q_star = np.zeros(plant.num_positions())
Ratfk = RationalForwardKinematics(plant)

diagram_vis_context = diagram.CreateDefaultContext()
plant_vis_context =  diagram.GetMutableSubsystemContext(plant, diagram_vis_context)
vis_bundle = vis_utils.VisualizationBundle(
    diagram, diagram_vis_context, plant, plant_vis_context,
    Ratfk, meshcat, q_star
)

In [ ]:
#compute limits in s-space
limits_s = []
lower_q = plant.GetPositionLowerLimits()
upper_q = plant.GetPositionUpperLimits()

for q in [lower_q,upper_q]:
    limits_s.append(Ratfk.ComputeSValue(np.array(q), q_star))
limits_s = np.array(limits_s)
print(limits_s)

In [ ]:
from pydrake.all import InverseKinematics
ik = InverseKinematics(plant, diagram.GetMutableSubsystemContext(plant, diagram_context))
min_dist = 1e-5
collision_constraint = ik.AddMinimumDistanceConstraint(
            min_dist, 1e-5)
diagram_col_context = diagram.CreateDefaultContext()
plant_col_context = diagram.GetMutableSubsystemContext(plant, diagram_col_context)
scene_graph_col_context = diagram.GetMutableSubsystemContext(scene_graph, diagram_col_context)
def check_collision_q_by_ik(q, min_dist=1e-5):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        return 1 - 1 * \
            float(collision_constraint.evaluator().CheckSatisfied(q, min_dist))
    else:
        return 1
def check_collision_s_by_ik(s, min_dist=1e-5):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_ik(q, min_dist)

def check_collision_q_by_query(q):
    if np.all(q >= plant.GetPositionLowerLimits()) and \
            np.all(q <= plant.GetPositionUpperLimits()):
        plant.SetPositions(plant_col_context, q)
        query_object = query_port.Eval(scene_graph_col_context)
        return 1 if query_object.HasCollisions() else 0
    else:
        return 1
    
def check_collision_s_by_query(s):
    s = np.array(s)
    q = Ratfk.ComputeQValue(s, q_star)
    return check_collision_q_by_query(q)

## Set up the sliders so we can move the plant around manually

In [ ]:
sliders = []
diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=plant.GetPositions(plant_context)[i],
                                       description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    plant.SetPositions(plant_context, q)
    diagram.ForcedPublish(diagram_context)

    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

In [ ]:
col_checker = StraightLineCollisionChecker(check_collision_s_by_query, 100)
def sample_col_free_point():
    q = np.random.uniform(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits())
    s = Ratfk.ComputeSValue(q, q_star)
    return s
np.random.seed(0)
prm = PRMFixedEdges(sample_col_free_point,
         100,
         col_checker,
         dist_thresh = 100)
prm.draw_tree(vis_bundle, end_effector)

In [ ]:
def make_line_polys(prm, max_num_edges = -1):
    
    polys = np.empty(shape=(plant.num_positions(), 
                            len(prm.prm.edges()) if max_num_edges < 0 else max_num_edges), dtype = object)
    for i, (s0, s1) in enumerate(prm.prm.edges()):
        for j in range(plant.num_positions()):
            if max_num_edges > 0 and not i < max_num_edges:
                break
            polys[j,i] = PolynomialCommon(np.array([s0[j],s1[j]-s0[j]]))
        
    return polys

path_safe = make_line_polys(prm,500)
# path_unsafe = make_line_polys(PRM_bad)
    

In [ ]:
# The object we will use to perform our certification.
t0 = time.time()
cspace_free_path = CspaceFreePath(plant, scene_graph, q_star,
                                  maximum_path_degree = 1, 
                                  plane_order = 1)
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")


cert_options = CspaceFreePath.FindSeparationCertificateGivenPathOptions()
cert_options.terminate_segment_certification_at_failure = False

cert_options.num_threads = -1
cert_options.verbose = False
cert_options.solver_id = MosekSolver.id()
cert_options.solver_options = SolverOptions()
cert_options.terminate_path_certification_at_failure = False


In [ ]:
print(len(cspace_free_path.separating_planes()))

In [ ]:
from pydrake.all import GeometrySet, CollisionFilterDeclaration
collision_filter_manager = scene_graph.collision_filter_manager()
pend_geom_set = GeometrySet(pend_geom_ids)
decl = CollisionFilterDeclaration().ExcludeWithin(pend_geom_set)
collision_filter_manager.Apply(decl)
t0 = time.time()
cspace_free_path2 = CspaceFreePath(plant, scene_graph, q_star,
                                  maximum_path_degree = 1, 
                                  plane_order = 1)
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")
print(len(cspace_free_path2.separating_planes()))

In [ ]:
ignored_collision_pairs = set()
for g1 in pend_geom_ids[:-1]:
    for g2 in pend_geom_ids[1:]: 
        ignored_collision_pairs.add((g1,g2))

In [ ]:
t0 = time.time()
statistics, cert_result = \
    cspace_free_path2.FindSeparationCertificateGivenPath(path_safe, 
                                                        ignored_collision_pairs,
                                                        cert_options)
t1 = time.time()
print(f"Certification of safe PRM in {t1-t0}s")

In [ ]:
solve_times = np.array([list(s.time_to_solve_prog.values()) for s in statistics])
build_times = np.array([list(s.time_to_build_prog.values()) for s in statistics])
total_times = np.array([list(s.total_time_to_certify_pair.values()) for s in statistics])


In [ ]:
fig, ax = plt.subplots(1,3)
ax[0].stairs(*np.histogram(solve_times))
ax[0].set_title('program solve times')
ax[1].stairs(*np.histogram(build_times))
ax[1].set_title('program build times')
ax[2].stairs(*np.histogram(total_times))
ax[2].set_title('pair certification time')

In [ ]:
plt.stairs(*np.histogram(solve_times))
plt.title(f'Distribution of solve times for pairs, {num_pend} links, {num_obstacles} obstacles')
plt.xlabel("ms")
plt.ylabel("number of pairs")

In [ ]:
import os
num_cores = os.cpu_count()
print(num_cores)
total_prog_solve_times = np.array([s.total_time_to_solve_progs()/num_cores for s in statistics])
plt.stairs(*np.histogram(total_prog_solve_times))
plt.title(f'Total Certification Times PRM Edges, {num_pend} links, {num_obstacles} obstacles {500} edges')
plt.xlabel("ms")
plt.ylabel("number of edges requiring time")

In [ ]:
import pickle
with open("tmp.pkl", "wb") as f:
    pickle.dump(statistics[0], f)

In [ ]:
with open("tmp.pkl", "rb") as f:
    tmp = pickle.load(f)

In [ ]:
total_prog_solve_times.shape

In [ ]:
def plot_pair_safety(pairs, statistics, body,
                                    path_size = 0.01, num_points = 1000, prefix = "prm"):
    meshcat.Delete(f"{prefix}_safe")
    meshcat.Delete(f"{prefix}_unsafe")
    
    safe_options = vis_utils.TrajectoryVisualizationOptions(
        path_color = Rgba(0,1,0,1), path_size = path_size, num_points = num_points
    )
    unsafe_options = vis_utils.TrajectoryVisualizationOptions(
        path_color = Rgba(1,0,0,1), path_size = path_size, num_points = num_points
    )
    
    for idx, (s0, s1) in enumerate(pairs):
        s0, s1 = np.array(s0), np.array(s1)
        if statistics[idx].certified_safe():
            vis_utils.visualize_s_space_segment(vis_bundle,s0, s1, body,
                                        f"{prefix}_safe/seg_{idx}", safe_options)
        else:
            vis_utils.visualize_s_space_segment(vis_bundle,s0, s1, body, 
                                        f"{prefix}_unsafe/seg_{idx}", unsafe_options)
plot_pair_safety(prm.prm.edges(), statistics, end_effector)

In [ ]:
path_safe.shape

In [ ]:
unsafe_path = None
for idx, stats in enumerate(statistics):
    if not stats.certified_safe():
        unsafe_path = path_safe[:, idx]
col_found = False
for t in np.linspace(0, 1, int(1e3)):
    s = np.array([unsafe_path[i].EvaluateUnivariate(t) for i in range(unsafe_path.shape[0])])
    if check_collision_s_by_query(s):
        print("COLLISION FOUND")
        col_found = True
        plant.SetPositions(plant_context, Ratfk.ComputeQValue(s, q_star))
        diagram.ForcedPublish(diagram_context)
        break
print(col_found)

In [ ]:
import numpy as np
num_links = np.arange(2, 12, 1)
num_boxes = np.array([10, 50, 100])
num_edges = np.array([50, 100, 1000])
good_checker = np.array([True, False])

NN, KK = np.meshgrid(num_links, num_boxes,  indexing = 'ij')
# LL,GG, NN,  KK   = np.meshgrid(num_edges,good_checker,num_links,  num_boxes, indexing = 'ij')
for i in range(NN.shape[0]):
    for j in range(NN.shape[1]):
                print(NN[i,j],KK[i,j])